In [37]:
import pandas as pd 
import re
def clean_title(title):
    return re.sub(r"\s*\(.*?\)", "", title)
pd.set_option('display.max_columns', None)

In [38]:
ratings = pd.read_csv("C:\\Users\\JaredPeck\\Documents\\reccomendation_system\\app\\csv\\ratings.csv")
books = pd.read_csv("C:\\Users\\JaredPeck\\Documents\\reccomendation_system\\app\\csv\\books.csv")

In [39]:
combined_df = books.merge(ratings,  on = "book_id")

In [40]:
cleaned_df =  combined_df[["book_id", 'original_publication_year', "title",   'authors', 'language_code', 'average_rating',
                     'ratings_count', 'work_ratings_count', 'work_text_reviews_count','user_id', 'rating', "image_url"]]
cleaned_df 

,book_id,original_publication_year,title,authors,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,user_id,rating,image_url
0,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,2886,5,https://images.gr-assets.com/books/1447303603m...
1,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,6158,5,https://images.gr-assets.com/books/1447303603m...
2,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,3991,4,https://images.gr-assets.com/books/1447303603m...
3,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,5281,5,https://images.gr-assets.com/books/1447303603m...
4,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,5721,5,https://images.gr-assets.com/books/1447303603m...
...,...,...,...,...,...,...,...,...,...,...,...,...
5976474,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,35336,4,https://images.gr-assets.com/books/1403194704m...
5976475,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,17999,3,https://images.gr-assets.com/books/1403194704m...
5976476,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,49007,4,https://images.gr-assets.com/books/1403194704m...
5976477,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,43319,5,https://images.gr-assets.com/books/1403194704m...


In [41]:
cleaned_df.shape

(5976479, 12)

In [42]:
x = cleaned_df["user_id"].value_counts() > 100
y= x[x].index

In [43]:
ratings_with_books = cleaned_df[cleaned_df['user_id'].isin(y)]
ratings_with_books

,book_id,original_publication_year,title,authors,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,user_id,rating,image_url
0,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,2886,5,https://images.gr-assets.com/books/1447303603m...
1,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,6158,5,https://images.gr-assets.com/books/1447303603m...
2,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,3991,4,https://images.gr-assets.com/books/1447303603m...
4,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,5721,5,https://images.gr-assets.com/books/1447303603m...
5,1,2008.0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,eng,4.34,4780653,4942365,155254,5034,5,https://images.gr-assets.com/books/1447303603m...
...,...,...,...,...,...,...,...,...,...,...,...,...
5976472,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,51328,1,https://images.gr-assets.com/books/1403194704m...
5976474,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,35336,4,https://images.gr-assets.com/books/1403194704m...
5976476,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,49007,4,https://images.gr-assets.com/books/1403194704m...
5976477,10000,1998.0,The First World War,John Keegan,NaN,4.00,9162,9700,364,43319,5,https://images.gr-assets.com/books/1403194704m...


In [44]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [45]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [46]:
number_rating.head()

,title,num_of_rating
0,"Angels (Walsh Family, #3)",199
1,#GIRLBOSS,120
2,'Salem's Lot,3804
3,"'Tis (Frank McCourt, #2)",600
4,"1,000 Places to See Before You Die",317


In [47]:
final_rating = ratings_with_books.merge(number_rating, on='title')
final_rating["title"].apply(lambda x: clean_title(x))
final_rating["title"].value_counts()

The Hunger Games (The Hunger Games, #1)                     16667
Harry Potter and the Sorcerer's Stone (Harry Potter, #1)    15922
To Kill a Mockingbird                                       14606
The Great Gatsby                                            12774
Catching Fire (The Hunger Games, #2)                        12360
                                                            ...  
شاب كشك في رحلة البحث عن الجادون                                1
ڤيرتيجو                                                         1
رغم الفراق                                                      1
محال                                                            1
المانيفستو                                                      1
Name: title, Length: 9940, dtype: int64

In [48]:
final_rating = final_rating[final_rating['num_of_rating'] >= 50]

In [49]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [50]:
frdf = pd.read_csv("final_df.csv")

In [51]:
book_pivot = frdf.pivot_table(columns='user_id', index='title', values= 'rating')

In [52]:
# always want to fill NA values with 0's before sparse matrix this bitch :) 
book_pivot = book_pivot.fillna(0)

In [53]:
book_pivot.index

Index([' Angels (Walsh Family, #3)', '#GIRLBOSS', ''Salem's Lot',
       ''Tis (Frank McCourt, #2)', '1,000 Places to See Before You Die',
       '10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works',
       '100 Bullets, Vol. 1: First Shot, Last Call', '100 Love Sonnets',
       '100 Selected Poems', '10th Anniversary (Women's Murder Club, #10)',
       ...
       'Zoo', 'Zorba the Greek', 'Zorro',
       'for colored girls who have considered suicide/when the rainbow is enuf',
       'god is Not Great: How Religion Poisons Everything', 'number9dream',
       'sTORI Telling', 'ttyl (Internet Girls, #1)',
       'xxxHolic, Vol. 1 (xxxHOLiC, #1)',
       '美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon Shinsōban 1]'],
      dtype='object', name='title', length=9181)

In [54]:
# pass in the csr_matrix to make it computationally faster
from scipy.sparse import csr_matrix

In [55]:
# add pivot table to the sparse matrix 
book_sparse = csr_matrix(book_pivot)

In [56]:
import pickle 
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')
model.fit(book_sparse)

with open('nearest_neighbors_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [57]:

from typing import List
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

def get_book_vectors(df: pd.DataFrame, book_titles: List[str]) -> np.ndarray:
    book_vectors = []
    for title in book_titles:
        if title in df.index:
            book_vectors.append(df.loc[title].values)
        else:
            raise ValueError(f"The book '{title}' is not in the DataFrame index.")
    return np.array(book_vectors)

def books_recommendation(user_liked_books: List[str], model: NearestNeighbors, df: pd.DataFrame, n_neighbors: int = 8) -> List[str]:
    book_vectors = get_book_vectors(df, user_liked_books)
    average_vector = np.mean(book_vectors, axis=0).reshape(1, -1)
    distance, suggestion = model.kneighbors(average_vector, n_neighbors=n_neighbors + len(user_liked_books))
    
    suggested_books = [df.index[suggestion[0][i]] for i in range(len(suggestion[0]))]
    
    filtered_suggestions = [book for book in suggested_books if book not in user_liked_books][:n_neighbors]
    
    return filtered_suggestions

user_liked_books = [
    'The Hunger Games (The Hunger Games, #1)',
    'Twilight (Twilight, #1)'
]

suggested_books = books_recommendation(user_liked_books, model, book_pivot, 9)
suggested_books


['New Moon (Twilight, #2)',
 'Eclipse (Twilight, #3)',
 'Catching Fire (The Hunger Games, #2)',
 'Mockingjay (The Hunger Games, #3)',
 'Breaking Dawn (Twilight, #4)',
 'The Host (The Host, #1)',
 'Divergent (Divergent, #1)',
 'City of Bones (The Mortal Instruments, #1)',
 'Fifty Shades of Grey (Fifty Shades, #1)']

In [61]:
with open("nearest_neighbors_model.pkl", "rb") as file:
    pickled_model = pickle.load(file)

suggested_books = books_recommendation(user_liked_books, pickled_model, book_pivot, 9)
suggested_books



['New Moon (Twilight, #2)',
 'Eclipse (Twilight, #3)',
 'Catching Fire (The Hunger Games, #2)',
 'Mockingjay (The Hunger Games, #3)',
 'Breaking Dawn (Twilight, #4)',
 'The Host (The Host, #1)',
 'Divergent (Divergent, #1)',
 'City of Bones (The Mortal Instruments, #1)',
 'Fifty Shades of Grey (Fifty Shades, #1)']